In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import OrdinalEncoder
from category_encoders import TargetEncoder 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

app_train = pd.read_csv('application_train.csv')
app_test = pd.read_csv('application_test.csv')
credcard = pd.read_csv('credit_aggregates.csv')
prev = pd.read_csv('previous_aggregates.csv')
installments = pd.read_csv('installments_aggregates.csv')

In [44]:
#
app_train.shape

(307511, 122)

In [45]:
#combined data
app = app_train.merge(prev, on="SK_ID_CURR", how='left').merge(installments, on='SK_ID_CURR', how='left').merge(credcard, on='SK_ID_CURR', how='left').fillna(0)
print(app.shape)
app.head()

(307511, 140)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,DAYS_LAST_DUE,DAYS_TERMINATION,TOTAL_ANNUITY,TOTAL_APPLICATION,TOTAL_PAYMENT,TOTAL_OVERDUE_DAYS,TOTAL_BALANCE,TOTAL_LIMIT,AVG_DIFF_RECEIV_BALANCE,AVG_RATIO_CASH
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,-25.0,-17.0,9251.775,179055.0,219625.695,-388.0,0.0,0.0,0.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,-536.0,-527.0,169661.970,1452573.0,1618864.650,-179.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,-724.0,-714.0,5357.250,20106.0,21288.465,-23.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,-151.0,-143.0,141907.050,2625259.5,1007153.415,-310.0,0.0,1620000.0,-270000.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,365243.0,365243.0,73672.830,999832.5,806127.975,-240.0,0.0,0.0,0.0,0.0


In [46]:
#imbalance
app['TARGET'].shape[0] - np.sum(app['TARGET'])
app = app.drop(columns=['CNT_FAM_MEMBERS','LIVE_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_WORK_REGION', 'OBS_60_CNT_SOCIAL_CIRCLE'])
print(app.shape)

(307511, 136)


In [47]:
#pre-process
#continuous
all_numerical_cols = list(app.select_dtypes(exclude='object').columns)
cont_cols = [col for col in all_numerical_cols if col != "TARGET" and col[:5]!='FLAG_']
# app_test = app_test.drop(columns=['CNT_FAM_MEMBERS','LIVE_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_WORK_REGION', 'OBS_60_CNT_SOCIAL_CIRCLE'])
cols_to_remove = ['AMT_CREDIT', 'CNT_FAM_MEMBERS', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'OBS_60_CNT_SOCIAL_CIRCLE','SK_ID_CURR']
cont= list(set(cont_cols) - set(cols_to_remove))
len(cont)

89

In [48]:

app['Employed/Birth'] = app['DAYS_EMPLOYED']/app['DAYS_BIRTH'] 
app['Flag_Greater_30'] = (app['DAYS_BIRTH']/-365.25).apply(lambda x: 1 if x > 30 else 0)
app['Flag_Employment_Greater_5'] = (app['DAYS_EMPLOYED']/-365.25).apply(lambda x: 1 if x > 5 else 0)


In [49]:
#set X1 and Y1 and try to train
cat_col = app.select_dtypes('object')
enc = TargetEncoder()
app[cat_col.columns] = enc.fit_transform(app[cat_col.columns], app['TARGET'])

In [50]:
#deal with the nas
#drop some columns
k = 0
for i in cat_col:
    if np.sum(app[i].isna().sum())/app.shape[0] >= 0.4:
        app = app.drop(columns=[i])
    app[i].fillna(app[i].mode(), inplace=True)
for i in cont_cols:
    if np.sum(app[i].isna().sum())/app.shape[0] >= 0.4:
        app = app.drop(columns=[i])
    app[i].fillna(app[i].median(), inplace=True)
    sc = StandardScaler().fit(app[[i]])
    app[i] = sc.transform(app[[i]])
for i in app.columns:
    if max(app[i])>5 and i not in cat_col:
        sc = StandardScaler().fit(app[[i]])
        app[i] = sc.transform(app[[i]])
print(app.shape)

(307511, 139)


In [51]:
#na checks zero!!!
app.isna().sum().sum()

0

In [52]:
app['NAME_CONTRACT_STATUS']

0         0.073674
1         0.073674
2         0.073674
3         0.073674
4         0.073674
            ...   
307506    0.073674
307507    0.073674
307508    0.073674
307509    0.073674
307510    0.124919
Name: NAME_CONTRACT_STATUS, Length: 307511, dtype: float64

In [53]:
#over sampling and onehot encode
def posi_extend(X, y):
    more_posi = (int) ((y.shape[0] - np.sum(y))*0.5)
    posi_idx = [i for i in range(len(y)) if y.iloc[i] == 1]
    indices = np.random.choice(posi_idx, size=more_posi, replace=True)
    X_ex = X.iloc[indices]
    y_ex = y.iloc[indices]
    X = pd.concat([X, X_ex], axis=0)
    y = pd.concat([y, y_ex], axis=0)
    return X, y

#two stages of training 
train1 = app.loc[:190000,:]
train2 = app.loc[190000:260000,:]
print(train1.shape, train2.shape)

Y1 = train1.iloc[:,1]
X1 = train1.drop(["TARGET", "SK_ID_CURR"],axis=1)
Y2 = train2.iloc[:,1]
X2 = train2.drop(["TARGET", "SK_ID_CURR"],axis=1)

X1, Y1 = posi_extend(X1, Y1)
X2, Y2 = posi_extend(X2, Y2)
print(X1.shape, X2.shape)

#final test set, imbalanced
test_data = app.loc[260000:,:]
YTest = test_data.iloc[:,1]
XTest = test_data.drop(["TARGET", "SK_ID_CURR"],axis=1)
print(XTest.shape)

(190001, 139) (70001, 139)
(277287, 137) (102180, 137)
(47511, 137)


In [54]:
X1.head(10)

,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,TOTAL_APPLICATION,TOTAL_PAYMENT,TOTAL_OVERDUE_DAYS,TOTAL_BALANCE,TOTAL_LIMIT,AVG_DIFF_RECEIV_BALANCE,AVG_RATIO_CASH,Employed/Birth,Flag_Greater_30,Flag_Employment_Greater_5
0,0.083459,0.101419,0.085002,0.079616,-0.577538,0.142129,-0.478095,-0.166065,-0.505662,0.081830,...,-0.494912,-0.468572,-0.114705,-0.249347,-0.366327,0.306694,-0.227351,0.067329,0,0
1,0.083459,0.069993,0.085002,0.083249,-0.577538,0.426792,1.725450,0.592710,1.600480,0.074946,...,0.376084,1.077965,0.299068,-0.249347,-0.366327,0.306694,-0.227351,0.070862,1,0
2,0.054783,0.101419,0.072437,0.079616,-0.577538,-0.427196,-1.152888,-1.404507,-1.090025,0.081830,...,-0.603622,-0.687788,0.607912,-0.249347,-0.366327,0.306694,-0.227351,0.011814,1,0
3,0.083459,0.069993,0.085002,0.079616,-0.577538,-0.142533,-0.711430,0.177929,-0.651753,0.081830,...,1.178119,0.401858,0.039717,-0.249347,1.786412,-2.385819,-0.227351,0.159905,1,1
4,0.083459,0.101419,0.085002,0.079616,-0.577538,-0.199466,-0.213734,-0.361658,-0.067390,0.081830,...,0.066442,0.179671,0.178301,-0.249347,-0.366327,0.306694,-0.227351,0.152418,1,1
5,0.083459,0.101419,0.085002,0.079616,-0.577538,-0.294354,-0.269648,0.028286,-0.225655,0.078716,...,-0.060765,0.347108,2.462960,-0.249347,-0.366327,0.306694,-0.227351,0.093737,1,0
6,0.083459,0.069993,0.072437,0.079616,0.807273,0.009287,2.389375,0.979238,2.318760,0.081830,...,-0.281589,-0.171952,-0.213694,-0.249347,-0.366327,0.306694,-0.227351,0.227174,1,1
7,0.083459,0.101419,0.072437,0.079616,-0.577538,0.806342,2.313036,1.032638,2.683987,0.081830,...,-0.438997,-0.407930,0.417854,-0.249347,-0.366327,0.306694,-0.227351,0.023820,1,0
8,0.083459,0.069993,0.085002,0.079616,-0.577538,-0.237421,1.044955,0.463557,1.016117,0.073768,...,0.098948,0.916475,1.461195,-0.249347,0.231656,-0.590810,-0.227351,-18.172198,1,0
9,0.054783,0.101419,0.085002,0.079616,-0.577538,-0.142533,-0.482064,-0.473114,-0.359571,0.081830,...,-0.414603,-0.156846,-1.751978,-0.249347,-0.366327,0.306694,-0.227351,0.139540,1,1


In [58]:

X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.3, random_state=42)
print("-------------------------RandomForestClassifier(random_state=42, max_depth=15)")
#clf = RandomForestClassifier(random_state=42, max_depth=15) 
clf = RandomForestClassifier(random_state=42, max_depth=15)
# clf = MLPClassifier(max_iter=100, hidden_layer_sizes=(512,128,32,32,32,16),alpha=0.1,learning_rate_init=0.001) #imbalanced test: AUC 0.662085296871909, F1 0.216791319919941
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(XTest)
print("auc: "+ str(roc_auc_score(YTest, y_pred[:,1])), "F1: "+str(f1_score(YTest, clf.predict(XTest))))

print("-------------------------DecisionTreeClassifier(random_state=42, max_depth=15)")
clf = DecisionTreeClassifier(random_state=42, max_depth=15)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(XTest)
print("auc: "+ str(roc_auc_score(YTest, y_pred[:,1])), "F1: "+str(f1_score(YTest, clf.predict(XTest))))

print("-------------------------XGBClassifier(max_depth=5)")
clf = XGBClassifier(max_depth=5)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(XTest)
print("auc: "+ str(roc_auc_score(YTest, y_pred[:,1])), "F1: "+str(f1_score(YTest, clf.predict(XTest))))

-------------------------RandomForestClassifier(random_state=42, max_depth=15)
auc: 0.7410044593032554 F1: 0.2439798125450613
-------------------------DecisionTreeClassifier(random_state=42, max_depth=15)
auc: 0.6059797279227317 F1: 0.22962503873566778
-------------------------XGBClassifier(max_depth=5)
auc: 0.7581080961135571 F1: 0.3049335199440168


In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.metrics import confusion_matrix

def linear_regression(X1, Y1, X2, y2, models, XTest, YTest, alpha=0.01, num_iterations=1000):
    
    y_hats = []
    #models train on predictions
    ms = []
    X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.3, random_state=42)
    for i in range(len(models)):
        m = models[i]
        # print(str(m)[:80])
        m.fit(X_train, y_train)
        print(roc_auc_score(y2, m.predict_proba(X2)[:,1]))
        y_hat = m.predict_proba(X2)[:,1]
        
        
        y_hats.append(y_hat)
        ms.append(m)
    y_hats = np.array(y_hats)
    lr = LogisticRegression()
    lr.fit(y_hats.T , y2)
    #检查回归方式是不是有问题
    print(lr.coef_)
    print(np.sqrt(mean_squared_error(y2, lr.predict(y_hats.T))))

    #pred
    input = []
    for m in ms:
        input.append(m.predict_proba(XTest)[:,1])
    input = np.array(input)
    y_pred = lr.predict_proba(input.T)[:,1]
    print(np.sum(lr.predict(input.T)))
    print(np.sum(YTest))

    return roc_auc_score(YTest, y_pred), f1_score(YTest, lr.predict(input.T)), confusion_matrix(YTest, lr.predict(input.T))


In [65]:
def lasso_reg(X1, Y1, X2, y2, models, XTest, YTest):
    y_hats = []
    #models train on predictions
    ms = []
    X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.3, random_state=42)
    for i in range(len(models)):
        m = models[i]
        # print(m)
        m.fit(X_train, y_train)
        print(roc_auc_score(y2, m.predict_proba(X2)[:,1]))
        y_hat = m.predict_proba(X2)[:,1]
        y_hats.append(y_hat)
        ms.append(m)
    y_hats = np.array(y_hats)
    lr = Lasso(alpha=0.01)
    lr.fit(y_hats.T , y2)
    print(lr.coef_)

    #pred
    input = []
    for m in ms:
        input.append(m.predict_proba(XTest)[:,1])
    input = np.array(input)
    z = lr.predict(input.T)
    # h = 1 / (1 + np.exp(-z))
    y_pred = z
    cate = (z > 0.5).astype(int)
    return roc_auc_score(YTest, y_pred),f1_score(YTest, cate)


In [66]:
#lasso test
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

models = [DecisionTreeClassifier(random_state=42,max_depth=5),
        XGBClassifier(max_depth=5, seed=42),
        XGBClassifier(max_depth=10, seed=42),
        XGBClassifier(max_depth=15, seed=42),
        MLPClassifier(hidden_layer_sizes=(16,16),max_iter=50,early_stopping=True)]
print("------------lasso:")
print(lasso_reg(X1, Y1, X2, Y2, models, XTest, YTest))
print("------------linear:")
print(linear_regression(X1, Y1, X2, Y2, models, XTest, YTest))

 

------------lasso:
0.7053870302106444
0.7585223183421224
0.7215461150485806
0.7281333425638858
0.7391792462851594
[0.         0.59575384 0.00421324 0.         0.17940482]
(0.7617303554286934, 0.3064949504308348)
------------linear:
0.7053870302106444
0.7585223183421224
0.7215461150485806
0.7281333425638858


/Library/Python/3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


0.7327859130796845
[[0.45386972 2.81817713 0.46605892 0.33761294 1.12754903]]
0.5288690630102654
8291
3754
(0.7645341750267062, 0.3081776670817767, array([[37322,  6435],
       [ 1898,  1856]]))


In [74]:
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

#aggregation without the bond K=136
models = [DecisionTreeClassifier(random_state=42,max_depth=5),
          MLPClassifier(hidden_layer_sizes=(10,100,10),max_iter=30,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(128,256,256,32),max_iter=50,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(128,32,32),max_iter=50,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(100,32),max_iter=50,alpha=0.1,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(128,128),max_iter=10,early_stopping=True),
        XGBClassifier(max_depth=15, learning_rate=0.1, n_estimators=100, seed=42),
        XGBClassifier(max_depth=10, learning_rate=0.01, n_estimators=100, seed=42),
        XGBClassifier(max_depth=25, learning_rate=0.01, n_estimators=100, seed=42),
        XGBClassifier(max_depth=50, learning_rate=0.1, n_estimators=1000, seed=42),
        MLPClassifier(hidden_layer_sizes=(10,100,10),max_iter=30,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(128,256,256,32),max_iter=50,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(128,32,32),max_iter=50,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(100,32),max_iter=50,alpha=0.1,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(128,128,128),max_iter=10,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(32,256,256,32),max_iter=10,early_stopping=True,learning_rate='adaptive',alpha=0.1),
        MLPClassifier(hidden_layer_sizes=(32,256,128,10),max_iter=10,early_stopping=True,learning_rate_init=0.01),
        MLPClassifier(hidden_layer_sizes=(32,128,128,32),max_iter=10,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(128,256,256,32),max_iter=50,learning_rate_init=0.01),
        MLPClassifier(hidden_layer_sizes=(32,64,64,32),max_iter=10,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(128,32,32,10),max_iter=10,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,64,128,32),max_iter=10,early_stopping=True,learning_rate_init=0.01,alpha=0.1),
        MLPClassifier(hidden_layer_sizes=(32,128,128,10),max_iter=10,early_stopping=True,learning_rate='adaptive',alpha=0.1),
        MLPClassifier(hidden_layer_sizes=(5,100,100),max_iter=10,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(5,5),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,10,1),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(5,100,100),max_iter=30,early_stopping=True,alpha=0.1),
        MLPClassifier(hidden_layer_sizes=(100,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,32),max_iter=50,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(5,100),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(30,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(32,128,128,16),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(8,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(32,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(80,),max_iter=30),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(128,16,16),max_iter=30,early_stopping=True,learning_rate='adaptive',alpha=0.1),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(500,),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(10,128,128),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(10,30),max_iter=50,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(10,32),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(5,32),max_iter=50,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(5,32,32),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,32,10),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,64),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,16),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(16,32),max_iter=30,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,32),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(64,),max_iter=30,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(64,5),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(64,10),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(64,16,16),max_iter=30,early_stopping=True,learning_rate='adaptive',alpha=0.1),
        MLPClassifier(hidden_layer_sizes=(32,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(8,8,8,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(8,16),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,8),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(8,32,32,8),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(64,8,8),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(128,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(16,16,128),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,64,64,64,16),max_iter=30,early_stopping=True,alpha=0.1),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(128,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,32,10),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,20,10),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(128,10),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(64,10),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(128,),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(100,10),max_iter=30,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,10),max_iter=50,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(128,32,32),max_iter=50,early_stopping=True,learning_rate='adaptive',alpha=0.1),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(32,8),max_iter=50,alpha=0.1),
        MLPClassifier(hidden_layer_sizes=(12,12),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(4,8,8),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,8,8),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(8,32,32),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(100,),max_iter=50,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(10,32,32,10),max_iter=50,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(8,32,32),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(100,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(256,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(32,32),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,32),max_iter=100),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(16,32,32),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(16,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(32,16),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(32,32),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(32,32),max_iter=100),
        MLPClassifier(hidden_layer_sizes=(16,16,10),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(256,10),max_iter=50,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(32,128,128,16),max_iter=100,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(128,),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(16,128),max_iter=50,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(64,32),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(64,64),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,),max_iter=50,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(10,32),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(128,),max_iter=100,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(16,),max_iter=50),
        MLPClassifier(hidden_layer_sizes=(32,),max_iter=30),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=30),
        MLPClassifier(hidden_layer_sizes=(64,10),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(128,),max_iter=50,early_stopping=True),
        MLPClassifier(hidden_layer_sizes=(10,),max_iter=50,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(32,16,16),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(128,),max_iter=50,early_stopping=True,learning_rate='adaptive'),
        MLPClassifier(hidden_layer_sizes=(16,32,64,),max_iter=50,early_stopping=True,learning_rate='adaptive')
]
print(linear_regression(X1, Y1, X2, Y2, models, XTest, YTest))
print(lasso_reg(X1, Y1, X2, Y2, models, XTest, YTest))

DecisionTreeClassifier(max_depth=5, random_state=42)
0.7021386816700232
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsa
0.764363105298356
DecisionTreeClassifier(max_depth=5, random_state=12)
0.7021386816700232
RandomForestClassifier(max_depth=15, random_state=42)
0.734737733782985
DecisionTreeClassifier(criterion='entropy', max_depth=20, min_samples_leaf=50,
 
0.6596407268330272
DecisionTreeClassifier(max_depth=5, max_features=50, random_state=123)
0.6988240469045279
DecisionTreeClassifier(max_depth=5, max_features=50, min_samples_leaf=50,
      
0.6908915699924272
DecisionTreeClassifier(max_depth=30, min_samples_leaf=50, random_state=333)
0.6563516040713042
DecisionTreeClassifier(max_depth=15, random_state=42)
0.6010761375655819
DecisionTreeClassifier(max_depth=25, max_features=10, random_state=1)
0.5560322398371429
DecisionTreeClassifier(max_depth=5, min_samples_leaf=100, random_state=12)
0.7021386816700232
DecisionTreeClassifier(max_depth=15, min_sam

/Library/Python/3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(


0.7316327084751513
MLPClassifier(early_stopping=True, hidden_layer_sizes=(128, 256, 256, 32),
     


/Library/Python/3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


0.6539525240491142
MLPClassifier(early_stopping=True, hidden_layer_sizes=(128, 32, 32),
           


...

0.7511557835353494
MLPClassifier(early_stopping=True, hidden_layer_sizes=(32, 16, 16),
              learning_rate='adaptive', max_iter=50)
0.7154266250623427
MLPClassifier(early_stopping=True, hidden_layer_sizes=(128,),
              learning_rate='adaptive', max_iter=50)


/Library/Python/3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


0.7059590736287586
MLPClassifier(early_stopping=True, hidden_layer_sizes=(16, 32, 64),
              learning_rate='adaptive', max_iter=50)


/Library/Python/3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


0.7142878998551196
[-0.          0.56848205 -0.         -0.          0.         -0.
 -0.          0.          0.00167547  0.         -0.          0.
  0.          0.05645391  0.          0.          0.          0.
 -0.          0.          0.          0.          0.          0.
  0.          0.00633605  0.         -0.         -0.          0.
  0.          0.          0.02253523  0.          0.          0.
  0.          0.          0.0874611   0.          0.          0.
 -0.          0.          0.          0.03239539  0.          0.
  0.05395341  0.00754705  0.          0.          0.          0.
  0.          0.          0.         -0.          0.00606208  0.
  0.          0.          0.00768253  0.          0.          0.
  0.04880298  0.          0.03456688  0.          0.          0.00163733
  0.          0.          0.          0.          0.          0.
  0.07119361  0.          0.          0.          0.          0.
  0.02537298  0.          0.          0.00077703  0.          0

In [ ]:
# Lasso (0.7678550349669907, 0.8247353244511797) > (0.76729769967911, 0.30869423640507976, array([[37123,  6634],      [ 1858,  1896]]))